In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import convolve, label, distance_transform_edt, binary_erosion, binary_dilation, gaussian_filter, zoom
from scipy.spatial import distance_matrix, ConvexHull
from scipy.stats import mode, entropy, skew, kurtosis
from scipy.signal import find_peaks, correlate2d
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA, NMF, FastICA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import hashlib
import itertools
import random
from functools import lru_cache
import warnings
warnings.filterwarnings('ignore')

class GeneticCodeEvolver:
    """Genetic algorithm for evolving code solutions"""
    
    def __init__(self, population_size=100, mutation_rate=0.3, crossover_rate=0.7):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.generation = 0
        
        # Code gene pool - building blocks for solutions
        self.gene_pool = {
            # Transformations
            'rot90': "[list(r)for r in zip(*g[::-1])]",
            'rot180': "[r[::-1]for r in g[::-1]]",
            'rot270': "[list(r)for r in zip(*g)][::-1]",
            'fliph': "[r[::-1]for r in g]",
            'flipv': "g[::-1]",
            'trans': "[list(r)for r in zip(*g)]",
            
            # Scaling
            'scale2x': "[[g[i//2][j//2]for j in range(len(g[0])*2)]for i in range(len(g)*2)]",
            'scale3x': "[[g[i//3][j//3]for j in range(len(g[0])*3)]for i in range(len(g)*3)]",
            'half': "[r[::2]for r in g[::2]]",
            
            # Color operations
            'swap_01': "[[1 if x==0 else 0 if x==1 else x for x in r]for r in g]",
            'inc_color': "[[(x+1)%10 for x in r]for r in g]",
            'dec_color': "[[(x-1)%10 for x in r]for r in g]",
            'max_color': "[[max(r)]*len(r)for r in g]",
            'min_color': "[[min(r)]*len(r)for r in g]",
            
            # Pattern operations
            'checker': "[[(i+j)%2 for j in range(len(g[0]))]for i in range(len(g))]",
            'diagonal': "[[1 if i==j else 0 for j in range(len(g[0]))]for i in range(len(g))]",
            'border': "[[1 if i in[0,len(g)-1]or j in[0,len(g[0])-1]else 0 for j in range(len(g[0]))]for i in range(len(g))]",
            
            # Position-based
            'pos_sum': "[[(i+j)%10 for j in range(len(g[0]))]for i in range(len(g))]",
            'pos_mul': "[[(i*j)%10 for j in range(len(g[0]))]for i in range(len(g))]",
            'pos_max': "[[max(i,j)%10 for j in range(len(g[0]))]for i in range(len(g))]",
            'pos_min': "[[min(i,j)%10 for j in range(len(g[0]))]for i in range(len(g))]",
            
            # Complex operations
            'outline': "[[g[i][j]if any(i+di<0 or i+di>=len(g)or j+dj<0 or j+dj>=len(g[0])or g[i+di][j+dj]!=g[i][j]for di,dj in[(0,1),(0,-1),(1,0),(-1,0)])else 0 for j in range(len(g[0]))]for i in range(len(g))]",
            'fill_holes': "[[g[i][j]if g[i][j]else max(g[i-1][j]if i>0 else 0,g[i][j-1]if j>0 else 0)for j in range(len(g[0]))]for i in range(len(g))]",
            
            # Conditional operations
            'if_zero': "[[0 if g[i][j]==0 else 1 for j in range(len(g[0]))]for i in range(len(g))]",
            'if_nonzero': "[[1 if g[i][j]!=0 else 0 for j in range(len(g[0]))]for i in range(len(g))]",
            
            # Aggregations
            'row_sum': "[[sum(r)%10]*len(r)for r in g]",
            'col_sum': "[[(sum(g[i][j]for i in range(len(g)))%10)for j in range(len(g[0]))]for _ in g]",
        }
        
        # Operation templates that can be combined
        self.templates = [
            "def p(g):return {op1}",
            "def p(g):h=len(g);w=len(g[0]);return {op1}",
            "def p(g):return {op1}if len(g)>5 else {op2}",
            "def p(g):t={op1};return {op2}",
            "def p(g):return[[(g[i][j]{cond})for j in range(len(g[0]))]for i in range(len(g))]",
            "def p(g):m={{k:v for k,v in {mapping}}};return[[m.get(x,x)for x in r]for r in g]",
        ]
        
        # Conditions for templates
        self.conditions = [
            "+1)%10",
            "*2)%10", 
            "if g[i][j]>0 else 0",
            "if i>j else g[i][j]",
            "if (i+j)%2 else 0",
            "+i+j)%10",
            "*i*j)%10",
        ]
        
    def create_individual(self):
        """Create a random individual (code solution)"""
        template = random.choice(self.templates)
        
        if '{op1}' in template and '{op2}' in template:
            op1 = random.choice(list(self.gene_pool.values()))
            op2 = random.choice(list(self.gene_pool.values()))
            code = template.format(op1=op1, op2=op2)
        elif '{op1}' in template:
            op1 = random.choice(list(self.gene_pool.values()))
            code = template.format(op1=op1)
        elif '{cond}' in template:
            cond = random.choice(self.conditions)
            code = template.format(cond=cond)
        elif '{mapping}' in template:
            # Create random mapping
            mapping = [(i, random.randint(0, 9)) for i in range(10)]
            code = template.format(mapping=mapping)
        else:
            code = template
            
        return code
    
    def mutate(self, individual):
        """Mutate an individual"""
        if random.random() < self.mutation_rate:
            # Various mutation strategies
            mutation_type = random.choice(['replace_gene', 'modify_value', 'add_operation', 'swap_template'])
            
            if mutation_type == 'replace_gene':
                # Replace a gene with another
                for old_gene, new_gene in [(k, v) for k, v in self.gene_pool.items()]:
                    if self.gene_pool[old_gene] in individual:
                        new_key = random.choice(list(self.gene_pool.keys()))
                        individual = individual.replace(self.gene_pool[old_gene], self.gene_pool[new_key])
                        break
                        
            elif mutation_type == 'modify_value':
                # Modify numeric values
                import re
                numbers = re.findall(r'\d+', individual)
                if numbers:
                    old_num = random.choice(numbers)
                    new_num = str((int(old_num) + random.randint(-2, 2)) % 10)
                    individual = individual.replace(old_num, new_num, 1)
                    
            elif mutation_type == 'add_operation':
                # Wrap with additional operation
                if 'return ' in individual:
                    op = random.choice(list(self.gene_pool.values()))
                    individual = individual.replace('return ', f'return {op[:-1]};g=', 1)
                    
        return individual
    
    def crossover(self, parent1, parent2):
        """Crossover two parents to create offspring"""
        if random.random() < self.crossover_rate:
            # Extract components from parents
            import re
            
            # Try to swap return statements
            return1 = re.search(r'return (.+?)(?:$|;)', parent1)
            return2 = re.search(r'return (.+?)(?:$|;)', parent2)
            
            if return1 and return2:
                # Swap return statements
                child1 = parent1.replace(return1.group(1), return2.group(1))
                child2 = parent2.replace(return2.group(1), return1.group(1))
                return child1, child2
                
        return parent1, parent2
    
    def evolve_population(self, population, fitness_scores):
        """Evolve population based on fitness scores"""
        # Sort by fitness
        sorted_pop = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
        
        # Keep top performers (elitism)
        elite_size = self.population_size // 5
        new_population = [ind for ind, _ in sorted_pop[:elite_size]]
        
        # Tournament selection for breeding
        while len(new_population) < self.population_size:
            # Tournament selection
            tournament_size = 5
            tournament = random.sample(sorted_pop, tournament_size)
            parent1 = max(tournament, key=lambda x: x[1])[0]
            
            tournament = random.sample(sorted_pop, tournament_size)
            parent2 = max(tournament, key=lambda x: x[1])[0]
            
            # Crossover
            child1, child2 = self.crossover(parent1, parent2)
            
            # Mutation
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            new_population.extend([child1, child2])
            
        self.generation += 1
        return new_population[:self.population_size]


class HierarchicalPatternMatcher:
    """Hierarchical pattern matching with weighted priorities"""
    
    def __init__(self):
        self.pattern_hierarchy = {
            # Level 1: Simple transformations (highest priority)
            'level1': {
                'patterns': ['identity', 'rotate', 'flip', 'transpose'],
                'weight': 1.0,
                'matchers': {
                    'identity': self.match_identity,
                    'rotate': self.match_rotation,
                    'flip': self.match_flip,
                    'transpose': self.match_transpose
                }
            },
            # Level 2: Scaling operations
            'level2': {
                'patterns': ['scale_up', 'scale_down', 'crop', 'pad'],
                'weight': 0.8,
                'matchers': {
                    'scale_up': self.match_scale_up,
                    'scale_down': self.match_scale_down,
                    'crop': self.match_crop,
                    'pad': self.match_pad
                }
            },
            # Level 3: Color operations
            'level3': {
                'patterns': ['color_map', 'color_swap', 'color_fill', 'color_extract'],
                'weight': 0.6,
                'matchers': {
                    'color_map': self.match_color_map,
                    'color_swap': self.match_color_swap,
                    'color_fill': self.match_color_fill,
                    'color_extract': self.match_color_extract
                }
            },
            # Level 4: Pattern operations
            'level4': {
                'patterns': ['pattern_fill', 'pattern_repeat', 'pattern_extract'],
                'weight': 0.4,
                'matchers': {
                    'pattern_fill': self.match_pattern_fill,
                    'pattern_repeat': self.match_pattern_repeat,
                    'pattern_extract': self.match_pattern_extract
                }
            },
            # Level 5: Complex operations
            'level5': {
                'patterns': ['composite', 'conditional', 'recursive'],
                'weight': 0.2,
                'matchers': {
                    'composite': self.match_composite,
                    'conditional': self.match_conditional,
                    'recursive': self.match_recursive
                }
            }
        }
        
        self.pattern_scores = defaultdict(float)
        
    def match_all_patterns(self, examples):
        """Match all patterns and return weighted scores"""
        matches = {}
        
        for level_name, level_data in self.pattern_hierarchy.items():
            level_weight = level_data['weight']
            
            for pattern_name in level_data['patterns']:
                matcher = level_data['matchers'][pattern_name]
                match_result = matcher(examples)
                
                if match_result['matched']:
                    score = match_result['confidence'] * level_weight
                    matches[pattern_name] = {
                        'score': score,
                        'level': level_name,
                        'details': match_result,
                        'code': match_result.get('code', None)
                    }
                    
        # Sort by score
        sorted_matches = sorted(matches.items(), key=lambda x: x[1]['score'], reverse=True)
        return sorted_matches
    
    def match_identity(self, examples):
        """Check if output equals input"""
        matched = all(np.array_equal(ex['input'], ex['output']) for ex in examples)
        return {
            'matched': matched,
            'confidence': 1.0 if matched else 0.0,
            'code': "def p(g):return g"
        }
    
    def match_rotation(self, examples):
        """Check for rotation patterns"""
        for k in [1, 2, 3]:
            matched = all(np.array_equal(np.rot90(np.array(ex['input']), k), 
                                       np.array(ex['output'])) for ex in examples)
            if matched:
                codes = {
                    1: "def p(g):return[list(r)for r in zip(*g[::-1])]",
                    2: "def p(g):return[r[::-1]for r in g[::-1]]",
                    3: "def p(g):return[list(r)for r in zip(*g)][::-1]"
                }
                return {
                    'matched': True,
                    'confidence': 1.0,
                    'rotation': k * 90,
                    'code': codes[k]
                }
        return {'matched': False, 'confidence': 0.0}
    
    def match_flip(self, examples):
        """Check for flip patterns"""
        # Horizontal flip
        matched_h = all(np.array_equal(np.fliplr(np.array(ex['input'])), 
                                     np.array(ex['output'])) for ex in examples)
        if matched_h:
            return {
                'matched': True,
                'confidence': 1.0,
                'direction': 'horizontal',
                'code': "def p(g):return[r[::-1]for r in g]"
            }
            
        # Vertical flip
        matched_v = all(np.array_equal(np.flipud(np.array(ex['input'])), 
                                     np.array(ex['output'])) for ex in examples)
        if matched_v:
            return {
                'matched': True,
                'confidence': 1.0,
                'direction': 'vertical',
                'code': "def p(g):return g[::-1]"
            }
            
        return {'matched': False, 'confidence': 0.0}
    
    def match_transpose(self, examples):
        """Check for transpose pattern"""
        matched = all(np.array_equal(np.array(ex['input']).T, 
                                   np.array(ex['output'])) for ex in examples)
        return {
            'matched': matched,
            'confidence': 1.0 if matched else 0.0,
            'code': "def p(g):return[list(r)for r in zip(*g)]"
        }
    
    def match_scale_up(self, examples):
        """Check for scale up patterns"""
        for scale in [2, 3, 4]:
            matched = all(
                np.array(ex['output']).shape[0] == np.array(ex['input']).shape[0] * scale and
                np.array(ex['output']).shape[1] == np.array(ex['input']).shape[1] * scale
                for ex in examples
            )
            if matched:
                return {
                    'matched': True,
                    'confidence': 0.9,
                    'scale': scale,
                    'code': f"def p(g):return[[g[i//{scale}][j//{scale}]for j in range(len(g[0])*{scale})]for i in range(len(g)*{scale})]"
                }
        return {'matched': False, 'confidence': 0.0}
    
    def match_scale_down(self, examples):
        """Check for scale down patterns"""
        for scale in [2, 3, 4]:
            matched = all(
                np.array(ex['output']).shape[0] == np.array(ex['input']).shape[0] // scale and
                np.array(ex['output']).shape[1] == np.array(ex['input']).shape[1] // scale
                for ex in examples
            )
            if matched:
                return {
                    'matched': True,
                    'confidence': 0.9,
                    'scale': scale,
                    'code': f"def p(g):return[r[::{scale}]for r in g[::{scale}]]"
                }
        return {'matched': False, 'confidence': 0.0}
    
    def match_crop(self, examples):
        """Check for cropping pattern"""
        # Simplified - check if output is smaller than input
        matched = all(
            np.array(ex['output']).shape[0] < np.array(ex['input']).shape[0] or
            np.array(ex['output']).shape[1] < np.array(ex['input']).shape[1]
            for ex in examples
        )
        if matched:
            # Try to find consistent crop boundaries
            out_h, out_w = np.array(examples[0]['output']).shape
            return {
                'matched': True,
                'confidence': 0.7,
                'code': f"def p(g):return[r[:{out_w}]for r in g[:{out_h}]]"
            }
        return {'matched': False, 'confidence': 0.0}
    
    def match_pad(self, examples):
        """Check for padding pattern"""
        matched = all(
            np.array(ex['output']).shape[0] > np.array(ex['input']).shape[0] or
            np.array(ex['output']).shape[1] > np.array(ex['input']).shape[1]
            for ex in examples
        )
        return {'matched': matched, 'confidence': 0.7 if matched else 0.0}
    
    def match_color_map(self, examples):
        """Check for color mapping pattern"""
        # Try to find consistent color mapping
        mapping = {}
        for ex in examples:
            inp = np.array(ex['input'])
            out = np.array(ex['output'])
            
            if inp.shape != out.shape:
                return {'matched': False, 'confidence': 0.0}
                
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if inp[i,j] in mapping:
                        if mapping[inp[i,j]] != out[i,j]:
                            return {'matched': False, 'confidence': 0.0}
                    else:
                        mapping[inp[i,j]] = out[i,j]
                        
        if mapping:
            return {
                'matched': True,
                'confidence': 0.8,
                'mapping': mapping,
                'code': f"def p(g):m={mapping};return[[m.get(x,x)for x in r]for r in g]"
            }
        return {'matched': False, 'confidence': 0.0}
    
    def match_color_swap(self, examples):
        """Check for color swap pattern"""
        # Check if exactly two colors are swapped
        for ex in examples:
            inp = np.array(ex['input'])
            out = np.array(ex['output'])
            
            if inp.shape != out.shape:
                return {'matched': False, 'confidence': 0.0}
                
            # Find changed colors
            unique_in = set(inp.flatten())
            unique_out = set(out.flatten())
            
            if len(unique_in) == 2 and unique_in == unique_out:
                a, b = sorted(unique_in)
                if all((inp == a) == (out == b) for ex in examples):
                    return {
                        'matched': True,
                        'confidence': 0.8,
                        'colors': (a, b),
                        'code': f"def p(g):return[[{b}if x=={a}else{a}if x=={b}else x for x in r]for r in g]"
                    }
                    
        return {'matched': False, 'confidence': 0.0}
    
    def match_color_fill(self, examples):
        """Check for color fill pattern"""
        # Check if output is single color
        for ex in examples:
            out = np.array(ex['output'])
            if len(np.unique(out)) == 1:
                color = out[0,0]
                return {
                    'matched': True,
                    'confidence': 0.7,
                    'color': color,
                    'code': f"def p(g):return[[{color}]*len(g[0])for _ in g]"
                }
        return {'matched': False, 'confidence': 0.0}
    
    def match_color_extract(self, examples):
        """Check for color extraction pattern"""
        # Check if output contains subset of input colors
        for color in range(10):
            matched = True
            for ex in examples:
                inp = np.array(ex['input'])
                out = np.array(ex['output'])
                
                if inp.shape != out.shape:
                    matched = False
                    break
                    
                # Check if output is input with only 'color' preserved
                mask = inp == color
                if not np.array_equal(out[mask], inp[mask]):
                    matched = False
                    break
                if not np.all(out[~mask] == 0):
                    matched = False
                    break
                    
            if matched:
                return {
                    'matched': True,
                    'confidence': 0.7,
                    'color': color,
                    'code': f"def p(g):return[[x if x=={color}else 0 for x in r]for r in g]"
                }
                
        return {'matched': False, 'confidence': 0.0}
    
    def match_pattern_fill(self, examples):
        """Check for pattern fill (checkerboard, stripes, etc)"""
        # Checkerboard pattern
        matched_checker = True
        for ex in examples:
            out = np.array(ex['output'])
            h, w = out.shape
            
            # Check if it's a checkerboard with any two colors
            colors = np.unique(out)
            if len(colors) == 2:
                c1, c2 = colors
                expected = np.array([[(i+j)%2 for j in range(w)] for i in range(h)])
                if not (np.array_equal(out, np.where(expected, c1, c2)) or
                       np.array_equal(out, np.where(expected, c2, c1))):
                    matched_checker = False
                    break
            else:
                matched_checker = False
                break
                
        if matched_checker:
            return {
                'matched': True,
                'confidence': 0.6,
                'pattern': 'checkerboard',
                'code': f"def p(g):return[[{c1}if(i+j)%2 else {c2} for j in range(len(g[0]))]for i in range(len(g))]"
            }
            
        return {'matched': False, 'confidence': 0.0}
    
    def match_pattern_repeat(self, examples):
        """Check for pattern repetition"""
        # Simplified check
        return {'matched': False, 'confidence': 0.0}
    
    def match_pattern_extract(self, examples):
        """Check for pattern extraction"""
        # Simplified check
        return {'matched': False, 'confidence': 0.0}
    
    def match_composite(self, examples):
        """Check for composite operations"""
        # This would check for combinations of simpler operations
        return {'matched': False, 'confidence': 0.0}
    
    def match_conditional(self, examples):
        """Check for conditional operations"""
        # This would check for if-then-else patterns
        return {'matched': False, 'confidence': 0.0}
    
    def match_recursive(self, examples):
        """Check for recursive patterns"""
        # This would check for recursive/iterative patterns
        return {'matched': False, 'confidence': 0.0}


class EnsembleVotingSystem:
    """Ensemble voting system for combining multiple predictions"""
    
    def __init__(self):
        self.voters = []
        self.weights = {}
        self.performance_history = defaultdict(list)
        
    def add_voter(self, voter_name, voter_func, initial_weight=1.0):
        """Add a voter to the ensemble"""
        self.voters.append({
            'name': voter_name,
            'func': voter_func,
            'weight': initial_weight
        })
        self.weights[voter_name] = initial_weight
        
    def vote(self, candidates, examples):
        """Combine votes from all voters"""
        vote_results = defaultdict(float)
        
        for voter in self.voters:
            voter_votes = voter['func'](candidates, examples)
            
            # Apply weights
            for candidate, score in voter_votes.items():
                vote_results[candidate] += score * voter['weight']
                
        # Normalize scores
        total_score = sum(vote_results.values())
        if total_score > 0:
            vote_results = {k: v/total_score for k, v in vote_results.items()}
            
        # Sort by score
        sorted_results = sorted(vote_results.items(), key=lambda x: x[1], reverse=True)
        return sorted_results
    
    def update_weights(self, voter_name, success):
        """Update voter weights based on performance"""
        self.performance_history[voter_name].append(1.0 if success else 0.0)
        
        # Calculate new weight based on recent performance
        if len(self.performance_history[voter_name]) >= 10:
            recent_performance = self.performance_history[voter_name][-20:]
            success_rate = sum(recent_performance) / len(recent_performance)
            
            # Update weight
            self.weights[voter_name] = 0.5 + success_rate  # Weight between 0.5 and 1.5
            
            # Update voter
            for voter in self.voters:
                if voter['name'] == voter_name:
                    voter['weight'] = self.weights[voter_name]
                    break


class MetaLearningOptimizer:
    """Meta-learning system to learn which strategies work for which problems"""
    
    def __init__(self):
        self.problem_features = []
        self.successful_strategies = []
        self.strategy_predictor = None
        self.feature_importance = None
        
    def extract_problem_features(self, examples):
        """Extract meta-features from the problem"""
        features = []
        
        # Size features
        inp_shape = np.array(examples[0]['input']).shape
        out_shape = np.array(examples[0]['output']).shape
        
        features.extend([
            inp_shape[0], inp_shape[1],
            out_shape[0], out_shape[1],
            out_shape[0] / inp_shape[0],  # Height ratio
            out_shape[1] / inp_shape[1],  # Width ratio
        ])
        
        # Color features
        inp_colors = set()
        out_colors = set()
        for ex in examples:
            inp_colors.update(np.array(ex['input']).flatten())
            out_colors.update(np.array(ex['output']).flatten())
            
        features.extend([
            len(inp_colors),
            len(out_colors),
            len(inp_colors & out_colors),  # Common colors
            len(out_colors - inp_colors),  # New colors
        ])
        
        # Transformation features
        same_size = all(np.array(ex['input']).shape == np.array(ex['output']).shape 
                       for ex in examples)
        features.append(1.0 if same_size else 0.0)
        
        # Pattern features
        has_symmetry = any(
            np.array_equal(np.array(ex['output']), np.fliplr(np.array(ex['output']))) or
            np.array_equal(np.array(ex['output']), np.flipud(np.array(ex['output'])))
            for ex in examples
        )
        features.append(1.0 if has_symmetry else 0.0)
        
        # Complexity features
        total_cells = sum(np.array(ex['input']).size + np.array(ex['output']).size 
                         for ex in examples)
        features.append(total_cells)
        
        return np.array(features)
    
    def record_success(self, problem_features, strategy_name):
        """Record a successful strategy for a problem"""
        self.problem_features.append(problem_features)
        self.successful_strategies.append(strategy_name)
        
        # Retrain predictor periodically
        if len(self.problem_features) >= 50 and len(self.problem_features) % 10 == 0:
            self.train_predictor()
            
    def train_predictor(self):
        """Train the strategy predictor"""
        from sklearn.preprocessing import LabelEncoder
        
        X = np.array(self.problem_features)
        
        # Encode strategy names
        le = LabelEncoder()
        y = le.fit_transform(self.successful_strategies)
        
        # Train ensemble classifier
        self.strategy_predictor = RandomForestClassifier(n_estimators=50, random_state=42)
        self.strategy_predictor.fit(X, y)
        
        # Calculate feature importance
        self.feature_importance = self.strategy_predictor.feature_importances_
        
        self.label_encoder = le
        
    def predict_best_strategies(self, problem_features, top_k=5):
        """Predict best strategies for a problem"""
        if self.strategy_predictor is None:
            return []
            
        # Get probability predictions
        probs = self.strategy_predictor.predict_proba([problem_features])[0]
        
        # Get top k strategies
        top_indices = np.argsort(probs)[-top_k:][::-1]
        top_strategies = [
            (self.label_encoder.inverse_transform([idx])[0], probs[idx])
            for idx in top_indices
        ]
        
        return top_strategies


class UltraAdvancedGeneticARCSolver:
    """Ultra-advanced solver with genetic algorithms, ensemble voting, and meta-learning"""
    
    def __init__(self):
        # Initialize all components
        self.genetic_evolver = GeneticCodeEvolver()
        self.pattern_matcher = HierarchicalPatternMatcher()
        self.ensemble_voter = EnsembleVotingSystem()
        self.meta_learner = MetaLearningOptimizer()
        
        # Pattern library (compact solutions)
        self.pattern_library = {
            'identity': ("def p(g):return g", 18),
            'rotate_90': ("def p(g):return[list(r)for r in zip(*g[::-1])]", 46),
            'rotate_180': ("def p(g):return[r[::-1]for r in g[::-1]]", 40),
            'rotate_270': ("def p(g):return[list(r)for r in zip(*g)][::-1]", 46),
            'flip_h': ("def p(g):return[r[::-1]for r in g]", 34),
            'flip_v': ("def p(g):return g[::-1]", 23),
            'transpose': ("def p(g):return[list(r)for r in zip(*g)]", 40),
            'scale_2x': ("def p(g):return[[g[i//2][j//2]for j in range(len(g[0])*2)]for i in range(len(g)*2)]", 88),
            'scale_3x': ("def p(g):return[[g[i//3][j//3]for j in range(len(g[0])*3)]for i in range(len(g)*3)]", 88),
            'half_size': ("def p(g):return[r[::2]for r in g[::2]]", 39),
        }
        
        # Initialize ensemble voters
        self._setup_voters()
        
        # Success tracking
        self.success_patterns = defaultdict(int)
        self.failure_patterns = defaultdict(int)
        
        # Model ensemble for complex patterns
        self.models = self._create_model_ensemble()
        
    def _setup_voters(self):
        """Setup voting functions for ensemble"""
        
        def pattern_match_voter(candidates, examples):
            """Vote based on pattern matching"""
            votes = {}
            for candidate in candidates:
                # Test candidate on examples
                score = self._test_candidate(candidate, examples)
                votes[candidate] = score
            return votes
        
        def length_voter(candidates, examples):
            """Vote based on code length (shorter is better)"""
            votes = {}
            max_len = max(len(c) for c in candidates) if candidates else 1
            for candidate in candidates:
                votes[candidate] = 1.0 - (len(candidate) / max_len)
            return votes
        
        def complexity_voter(candidates, examples):
            """Vote based on code complexity"""
            votes = {}
            for candidate in candidates:
                # Simple complexity measure
                complexity = candidate.count('for') + candidate.count('if') + candidate.count('lambda')
                votes[candidate] = 1.0 / (1 + complexity)
            return votes
        
        def historical_voter(candidates, examples):
            """Vote based on historical success"""
            votes = {}
            for candidate in candidates:
                # Check similarity to successful patterns
                success_score = 0
                for pattern, count in self.success_patterns.items():
                    if pattern in candidate:
                        success_score += count
                        
                failure_score = 0
                for pattern, count in self.failure_patterns.items():
                    if pattern in candidate:
                        failure_score += count
                        
                votes[candidate] = success_score / (1 + failure_score)
            return votes
        
        # Add voters to ensemble
        self.ensemble_voter.add_voter('pattern_match', pattern_match_voter, 2.0)
        self.ensemble_voter.add_voter('length', length_voter, 1.0)
        self.ensemble_voter.add_voter('complexity', complexity_voter, 0.5)
        self.ensemble_voter.add_voter('historical', historical_voter, 1.5)
        
    def _create_model_ensemble(self):
        """Create ensemble of ML models for complex pattern recognition"""
        return {
            'rf_classifier': RandomForestClassifier(n_estimators=100, random_state=42),
            'gb_classifier': GradientBoostingClassifier(n_estimators=50, random_state=42),
            'et_classifier': ExtraTreesClassifier(n_estimators=100, random_state=42),
            'mlp_classifier': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42),
            'knn_classifier': KNeighborsClassifier(n_neighbors=5),
            'svm_classifier': SVC(probability=True, random_state=42),
        }
        
    def _test_candidate(self, candidate, examples):
        """Test a candidate solution on examples"""
        try:
            # Create function from candidate code
            namespace = {}
            exec(candidate, namespace)
            p = namespace.get('p')
            
            if p is None:
                return 0.0
                
            # Test on examples
            correct = 0
            for ex in examples:
                try:
                    result = p([row[:] for row in ex['input']])
                    if result == ex['output']:
                        correct += 1
                except:
                    pass
                    
            return correct / len(examples) if examples else 0.0
            
        except:
            return 0.0
            
    def generate_candidates_genetic(self, examples, generations=20):
        """Generate candidates using genetic algorithm"""
        # Initialize population
        population = [self.genetic_evolver.create_individual() 
                     for _ in range(self.genetic_evolver.population_size)]
        
        best_candidates = []
        
        for gen in range(generations):
            # Evaluate fitness
            fitness_scores = []
            for individual in population:
                score = self._test_candidate(individual, examples)
                fitness_scores.append(score)
                
            # Track best candidates
            best_idx = np.argmax(fitness_scores)
            if fitness_scores[best_idx] > 0:
                best_candidates.append(population[best_idx])
                
            # Evolve population
            population = self.genetic_evolver.evolve_population(population, fitness_scores)
            
            # Early stopping if perfect solution found
            if max(fitness_scores) == 1.0:
                break
                
        return list(set(best_candidates))  # Remove duplicates
        
    def generate_candidates_pattern(self, examples):
        """Generate candidates using pattern matching"""
        candidates = []
        
        # Try hierarchical pattern matching
        matches = self.pattern_matcher.match_all_patterns(examples)
        
        for pattern_name, match_data in matches[:10]:  # Top 10 matches
            if match_data['details'].get('code'):
                candidates.append(match_data['details']['code'])
                
        # Add library patterns
        for name, (code, _) in self.pattern_library.items():
            candidates.append(code)
            
        return candidates
        
    def generate_candidates_learned(self, examples):
        """Generate candidates using learned patterns"""
        candidates = []
        
        # Extract problem features
        problem_features = self.meta_learner.extract_problem_features(examples)
        
        # Get predicted best strategies
        if self.meta_learner.strategy_predictor is not None:
            predicted_strategies = self.meta_learner.predict_best_strategies(problem_features)
            
            for strategy_name, confidence in predicted_strategies:
                if strategy_name in self.pattern_library:
                    candidates.append(self.pattern_library[strategy_name][0])
                    
        return candidates
        
    def generate_candidates_composite(self, examples):
        """Generate composite candidates by combining simpler operations"""
        candidates = []
        base_ops = [
            "g", 
            "[r[::-1]for r in g]",
            "g[::-1]",
            "[list(r)for r in zip(*g)]",
            "[[g[i//2][j//2]for j in range(len(g[0])*2)]for i in range(len(g)*2)]",
        ]
        
        # Try combinations
        for op1 in base_ops:
            for op2 in base_ops:
                if op1 != op2:
                    # Chain operations
                    candidates.append(f"def p(g):g={op1};return {op2}")
                    
        return candidates[:20]  # Limit number
        
    def generate_optimized_solution(self, task_data):
        """Generate optimized solution using all techniques"""
        examples = task_data['train']
        
        # Generate candidates from multiple sources
        all_candidates = []
        
        # 1. Pattern-based candidates
        pattern_candidates = self.generate_candidates_pattern(examples)
        all_candidates.extend(pattern_candidates)
        
        # 2. Genetic algorithm candidates
        genetic_candidates = self.generate_candidates_genetic(examples, generations=10)
        all_candidates.extend(genetic_candidates)
        
        # 3. Learned pattern candidates
        learned_candidates = self.generate_candidates_learned(examples)
        all_candidates.extend(learned_candidates)
        
        # 4. Composite candidates
        composite_candidates = self.generate_candidates_composite(examples)
        all_candidates.extend(composite_candidates)
        
        # Remove duplicates
        all_candidates = list(set(all_candidates))
        
        # Vote on best candidate
        if all_candidates:
            vote_results = self.ensemble_voter.vote(all_candidates, examples)
            
            # Test top candidates
            for candidate, vote_score in vote_results[:10]:
                test_score = self._test_candidate(candidate, examples)
                
                if test_score == 1.0:  # Perfect score
                    # Record success
                    problem_features = self.meta_learner.extract_problem_features(examples)
                    self.meta_learner.record_success(problem_features, 'genetic' if candidate in genetic_candidates else 'pattern')
                    
                    # Update success patterns
                    self.success_patterns[candidate[:20]] += 1
                    
                    return candidate
                    
        # Fallback solutions
        fallback_solutions = [
            "def p(g):return g",
            "def p(g):return[r[::-1]for r in g[::-1]]",
            "def p(g):return[list(r)for r in zip(*g[::-1])]",
            "def p(g):return[[0]*len(g[0])for _ in g]",
            "def p(g):return[[1]*len(g[0])for _ in g]",
        ]
        
        # Test fallbacks
        for fallback in fallback_solutions:
            if self._test_candidate(fallback, examples) > 0.5:
                return fallback
                
        # Ultimate fallback
        return "def p(g):return g"
        
    def analyze_and_learn(self, results):
        """Analyze results and update learning components"""
        # Update voter weights based on performance
        for task_id, (code, success) in results.items():
            if 'pattern' in code:
                self.ensemble_voter.update_weights('pattern_match', success)
            elif len(code) < 50:
                self.ensemble_voter.update_weights('length', success)
                
        # Update pattern success/failure counts
        for task_id, (code, success) in results.items():
            pattern_key = code[:30]  # Use first 30 chars as pattern key
            if success:
                self.success_patterns[pattern_key] += 1
            else:
                self.failure_patterns[pattern_key] += 1


def create_ultra_advanced_genetic_submission():
    """Create submission using ultra-advanced genetic approach"""
    solver = UltraAdvancedGeneticARCSolver()
    solutions = {}
    results = {}
    
    print("🧬 Generating Ultra-Advanced Genetic ARC Solutions...")
    print("🎯 Using Genetic Algorithms, Ensemble Voting, and Meta-Learning")
    print("=" * 70)
    
    successful = 0
    total_bytes = 0
    strategy_usage = defaultdict(int)
    
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        task_file = f"/kaggle/input/google-code-golf-2025/task{task_id}.json"
        
        try:
            with open(task_file) as f:
                task_data = json.load(f)
                
            # Generate optimized solution
            code = solver.generate_optimized_solution(task_data)
            solutions[task_id] = code
            
            # Verify solution
            try:
                namespace = {}
                exec(code, namespace)
                
                # Test on examples
                valid = True
                for ex in task_data['train'][:3]:
                    p = namespace['p']
                    result = p([row[:] for row in ex['input']])
                    if result != ex['output']:
                        valid = False
                        break
                        
                if valid:
                    successful += 1
                    status = "✅"
                    results[task_id] = (code, True)
                else:
                    status = "❌"
                    results[task_id] = (code, False)
            except:
                status = "⚠️"
                results[task_id] = (code, False)
                
            bytes_count = len(code)
            total_bytes += bytes_count
            
            # Progress update
            if task_num % 25 == 0:
                print(f"Progress: {task_num}/400 | Success rate: {successful/task_num:.1%} | "
                      f"Avg bytes: {total_bytes/task_num:.1f}")
                
                # Analyze and learn from results
                solver.analyze_and_learn(results)
                
        except Exception as e:
            code = "def p(g):return g"
            solutions[task_id] = code
            total_bytes += len(code)
            
    print(f"\n{'='*70}")
    print(f"✅ Completed: {successful}/400 valid solutions ({successful/400:.1%})")
    print(f"📊 Total bytes: {total_bytes:,}")
    print(f"📈 Average bytes per solution: {total_bytes/400:.1f}")
    
    # Final analysis
    print(f"\n🧬 Genetic Evolution Statistics:")
    print(f"  Final generation: {solver.genetic_evolver.generation}")
    print(f"  Pattern library size: {len(solver.pattern_library)}")
    print(f"  Successful patterns: {len(solver.success_patterns)}")
    
    print(f"\n🗳️ Ensemble Voting Weights:")
    for voter in solver.ensemble_voter.voters:
        print(f"  {voter['name']}: {voter['weight']:.2f}")
        
    # Create submission
    os.makedirs("submission", exist_ok=True)
    
    for task_id, code in solutions.items():
        with open(f"submission/task{task_id}.py", "w") as f:
            f.write(code)
            
    with zipfile.ZipFile("submission.zip", "w") as zipf:
        for task_id in solutions:
            zipf.write(f"submission/task{task_id}.py", f"task{task_id}.py")
            
    print(f"\n✨ Ultra-Advanced Genetic submission created: submission.zip")
    
    return solutions

if __name__ == "__main__":
    create_ultra_advanced_genetic_submission()